In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("México sin atipicos.csv")
df.head(5)

,Unnamed: 0,id,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,...,property_type,room_type,bathrooms_text,amenities,calendar_updated,has_availability,calendar_last_scraped,last_review,license,instant_bookable
0,0,35797.0,1.0,1.00,1.0,1.0,2.0,1.0,1.0,1.0,...,Entire villa,Entire home/apt,1 bath,"[""Free parking on premises"", ""Washer"", ""Wifi"",...",SIN INFORMACION,t,25/09/2024,SIN INFORMACION,SIN INFORMACION,f
1,1,44616.0,1.0,0.93,12.0,13.0,2.9,0.0,1.4,0.0,...,Entire home,Entire home/apt,5.5 baths,"[""Iron"", ""Washer"", ""Wifi"", ""Host greets you"", ...",SIN INFORMACION,t,26/09/2024,26/03/2023,SIN INFORMACION,f
2,2,67703.0,1.0,1.00,3.0,4.0,4.0,1.0,2.0,2.0,...,Entire rental unit,Entire home/apt,1 bath,"[""Iron"", ""Wifi"", ""Bathtub"", ""Refrigerator"", ""D...",SIN INFORMACION,t,26/09/2024,05/05/2024,SIN INFORMACION,f
3,3,70644.0,1.0,1.00,3.0,4.0,2.0,1.0,1.0,1.0,...,Entire rental unit,Entire home/apt,1 bath,"[""Iron"", ""Washer"", ""Outdoor furniture"", ""Wifi""...",SIN INFORMACION,t,25/09/2024,22/09/2024,SIN INFORMACION,f
4,4,107078.0,1.0,1.00,5.0,7.0,2.0,1.5,1.0,1.0,...,Entire loft,Entire home/apt,1.5 baths,"[""Iron"", ""Washer"", ""Outdoor furniture"", ""Wifi""...",SIN INFORMACION,t,25/09/2024,16/08/2023,SIN INFORMACION,f


In [3]:
df= df.drop(['Unnamed: 0'], axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26582 entries, 0 to 26581
Data columns (total 50 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           26582 non-null  float64
 1   host_response_rate           26582 non-null  float64
 2   host_acceptance_rate         26582 non-null  float64
 3   host_listings_count          26582 non-null  float64
 4   host_total_listings_count    26582 non-null  float64
 5   accommodates                 26582 non-null  float64
 6   bathrooms                    26582 non-null  float64
 7   bedrooms                     26582 non-null  float64
 8   beds                         26582 non-null  float64
 9   price                        26582 non-null  float64
 10  minimum_nights               26582 non-null  float64
 11  maximum_nights               26582 non-null  int64  
 12  availability_30              26582 non-null  int64  
 13  availability_60 

In [5]:
valores_nulos=df.isnull().sum()
valores_nulos

id                             0
host_response_rate             0
host_acceptance_rate           0
host_listings_count            0
host_total_listings_count      0
accommodates                   0
bathrooms                      0
bedrooms                       0
beds                           0
price                          0
minimum_nights                 0
maximum_nights                 0
availability_30                0
availability_60                0
availability_365               0
number_of_reviews              0
number_of_reviews_ltm          0
number_of_reviews_l30d         0
review_scores_rating           0
review_scores_accuracy         0
review_scores_cleanliness      0
review_scores_checkin          0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
reviews_per_month              0
last_scraped                   0
name                           0
description                    0
neighborhood_overview          0
picture_ur

In [6]:
# Convertir host_response_time a binario
df["host_response_time"] = df["host_response_time"].replace({
    "within an hour": "Rápido",
    "within a few hours": "Rápido",
    "within a day": "Lento",
    "a few days or more": "Lento",
    "SIN INFORMACION": "Lento"
})

# Verificar que la conversión fue correcta
print(df["host_response_time"].value_counts())

host_response_time
Rápido    20749
Lento      5833
Name: count, dtype: int64


In [7]:
df["host_is_superhost"] = df["host_is_superhost"].replace({"t": 1, "f": 0})
df["host_is_superhost"] = df["host_is_superhost"].replace({"SIN INFORMACION": 0})

C:\Users\angie\AppData\Local\Temp\ipykernel_18540\1378006797.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["host_is_superhost"] = df["host_is_superhost"].replace({"SIN INFORMACION": 0})


In [8]:
vars_indep=df[['host_acceptance_rate', 'host_is_superhost', 'number_of_reviews']]
var_dep=df['host_response_time']

In [9]:
X=vars_indep
y=var_dep

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [11]:
escalar=StandardScaler()

In [12]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [13]:
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

In [14]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [15]:
y_pred=algoritmo.predict(X_test)
y_pred

array(['Rápido', 'Rápido', 'Rápido', ..., 'Rápido', 'Rápido', 'Rápido'],
      dtype=object)

In [16]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[   0 1702]
 [   0 6273]]


In [17]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="Rápido")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.7865830721003135


In [18]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7865830721003135


In [19]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="Rápido")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


In [20]:
df["host_is_superhost"] = df["host_is_superhost"].replace({1:"t", 0:"f"})

In [21]:
vars_indep=df[['host_response_rate', 'host_acceptance_rate', 'host_total_listings_count']]
var_dep=df['host_is_superhost']

X=vars_indep
y=var_dep

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], dtype=object)

In [23]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[4930   90]
 [2872   83]]


In [24]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.6318892591643168


In [25]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.62858934169279


In [26]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9820717131474104


In [37]:
df["room_type"] = df["room_type"].replace({"Entire home/apt": 0, "Private room": 1, "Hotel room":2, "Shared room":3})

C:\Users\angie\AppData\Local\Temp\ipykernel_18540\1760127900.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["room_type"] = df["room_type"].replace({"Entire home/apt": 0, "Private room": 1, "Hotel room":2, "Shared room":3})


In [38]:
df["beds"] = df["beds"].replace({
    0: "De 0 a 2 camas",
    1: "De 0 a 2 camas",
    1.4: "De 0 a 2 camas",
    2: "De 2 a 3 camas",
    3: "De 2 a 3 camas"
})

# Verificar que la conversión fue correcta
print(df["beds"].value_counts())

beds
De 0 a 2 camas    17281
De 2 a 3 camas     9301
Name: count, dtype: int64


In [39]:
vars_indep=df[['accommodates', 'bedrooms', 'room_type']]
var_dep=df['beds']

X=vars_indep
y=var_dep

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['De 0 a 2 camas', 'De 0 a 2 camas', 'De 0 a 2 camas', ...,
       'De 0 a 2 camas', 'De 0 a 2 camas', 'De 0 a 2 camas'], dtype=object)

In [41]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[4428  714]
 [ 932 1901]]


In [43]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="De 0 a 2 camas")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.8261194029850746


In [44]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7936050156739812


In [45]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="De 0 a 2 camas")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.8611435239206534


In [50]:
df["host_is_superhost"] = df["host_is_superhost"].replace({"t": 1, "f": 0, "SIN INFORMACION":0})
df["host_verifications"] = df["host_verifications"].replace({
    "[]": 0, 
    "['phone']": 1, 
    "['email']": 2, 
    "['phone', 'work_email']": 3,
    "['email', 'work_email']": 4,
    "['email', 'phone']": 5,
    "['email', 'phone', 'work_email']": 6,
    "['work_email']": 7
})

C:\Users\angie\AppData\Local\Temp\ipykernel_18540\1410543520.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["host_is_superhost"] = df["host_is_superhost"].replace({"t": 1, "f": 0, "SIN INFORMACION":0})


In [51]:
vars_indep=df[['host_listings_count', 'host_verifications', 'host_is_superhost']]
var_dep=df['host_has_profile_pic']

X=vars_indep
y=var_dep

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [53]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[   0  150]
 [   0 7825]]


In [54]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.9811912225705329


In [55]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9811912225705329


In [56]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


In [63]:
vars_indep=df[['host_is_superhost', 'review_scores_communication', 'review_scores_rating']]
var_dep=df['host_identity_verified']

In [64]:
X=vars_indep
y=var_dep

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [66]:
escalar=StandardScaler()

In [67]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [68]:
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

In [69]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [70]:
y_pred=algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [71]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[   0  364]
 [   0 7611]]


In [72]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.9543573667711599


In [73]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9543573667711599


In [74]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


In [3]:
df["property_type"] = df["property_type"].replace({
    "Entire villa": 1,
    "Entire home": 2,
    "Entire rental unit": 3,
    "Entire loft": 4,
    "Private room": 5,
    "Private room in rental unit": 6,
    "Private room in home": 7,
    "Private room in condo": 8,
    "Private room in guest suite": 9,
    "Entire condo": 10,
    "Private room in serviced apartment": 11,
    "Room in serviced apartment": 12,
    "Entire guest suite": 13,
    "Private room in townhouse": 14,
    "Room in boutique hotel": 15,
    "Private room in bed and breakfast": 16,
    "Shared room in rental unit": 17,
    "Entire serviced apartment": 18,
    "Tiny home": 19,
    "Entire guesthouse": 20,
    "Private room in guesthouse": 21,
    "Private room in loft": 22,
    "Entire bungalow": 23,
    "Casa particular": 24,
    "Shared room in loft": 25,
    "Entire townhouse": 26,
    "Private room in tiny home": 27,
    "Shared room in guesthouse": 28,
    "Room in bed and breakfast": 29,
    "Shared room in hostel": 30,
    "Private room in farm stay": 31,
    "Shared room in home": 32,
    "Private room in villa": 33,
    "Entire cabin": 34,
    "Private room in casa particular": 35,
    "Room in hotel": 36,
    "Private room in floor": 37,
    "Private room in cabin": 38,
    "Shared room in guest suite": 39,
    "Room in aparthotel": 40,
    "Entire place": 41,
    "Private room in hostel": 42,
    "Earthen home": 43,
    "Entire hostel": 44,
    "Private room in earthen home": 45,
    "Private room in dorm": 46,
    "Private room in hut": 47,
    "Entire cottage": 48,
    "Entire chalet": 49,
    "Shared room in dorm": 50,
    "Entire in-law": 51,
    "Holiday park": 52,
    "Private room in dome": 53,
    "Shared room in condo": 54,
    "Campsite": 55,
    "Private room in vacation home": 56,
    "Room in casa particular": 57,
    "Castle": 58,
    "Private room in barn": 59,
    "Shared room in hotel": 60,
    "Farm stay": 61,
    "Shared room in tiny home": 62,
    "Hut": 63,
    "Shared room in cabin": 64,
    "Private room in cottage": 65,
    "Entire home/apt": 66,
    "Room in hostel": 67,
    "Shared room in serviced apartment": 68,
    "Entire vacation home": 69,
    "Shared room in tent": 70,
    "Shared room in bed and breakfast": 71,
    "Private room in shipping container": 72,
    "Private room in nature lodge": 73,
    "Private room in tower": 74,
    "Private room in lighthouse": 75,
    "Shared room in casa particular": 76,
    "Shared room in townhouse": 77,
    "Shared room in farm stay": 78,
    "Shared room in boutique hotel": 79,
    "Shipping container": 80,
    "Private room in pension": 81,
    "Private room in bungalow": 82,
    "Dome": 83,
    "Tent": 84,
    "Private room in chalet": 85,
    "Private room in tent": 86,
    "Private room in castle": 87,
    "Private room in houseboat": 88,
    "Boat": 89,
    "Private room in resort": 90,
    "Private room in minsu": 91,
    "Tower": 92
})


C:\Users\angie\AppData\Local\Temp\ipykernel_16160\3533744805.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["property_type"] = df["property_type"].replace({


In [4]:
df["room_type"] = df["room_type"].replace({
    "Entire home/apt":"Entire home/apt ",
    "Private room":"No Entire home/apt",
    "Shared room":"No Entire home/apt",
    "Hotel room":"No Entire home/apt"
})

print(df["room_type"].value_counts())

room_type
Entire home/apt       17363
No Entire home/apt     9219
Name: count, dtype: int64


In [5]:
vars_indep=df[['property_type', 'accommodates', 'price']]
var_dep=df['room_type']

X=vars_indep
y=var_dep

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['No Entire home/apt', 'No Entire home/apt', 'No Entire home/apt',
       ..., 'No Entire home/apt', 'No Entire home/apt',
       'Entire home/apt '], dtype=object)

In [7]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[4693  525]
 [ 731 2026]]


In [8]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="No Entire home/apt")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.7941983535868287


In [9]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8425078369905956


In [10]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="No Entire home/apt")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.7348567283278926


In [11]:
df["host_is_superhost"] = df["host_is_superhost"].replace({"SIN INFORMACION":'f'})

In [15]:
df["has_availability"] = df["has_availability"].replace({"SIN INFORMACION":'f'})

In [16]:
vars_indep=df[['price', 'availability_365', 'number_of_reviews']]
var_dep=df['has_availability']

X=vars_indep
y=var_dep

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [18]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[   0  314]
 [   0 7661]]


In [19]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.9606269592476488


In [20]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9606269592476488


In [21]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


In [22]:
df["host_response_time"] = df["host_response_time"].replace({
    "within an hour": 1,
    "within a few hours": 2,
    "within a day": 3,
    "a few days or more": 4,
    "SIN INFORMACION": 5
})

# Verificar que la conversión fue correcta
print(df["host_response_time"].value_counts())

host_response_time
1    18792
5     3622
2     1957
4     1108
3     1103
Name: count, dtype: int64


C:\Users\angie\AppData\Local\Temp\ipykernel_16160\2354697535.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["host_response_time"] = df["host_response_time"].replace({


In [23]:
vars_indep=df[['maximum_nights', 'minimum_nights', 'host_response_time']]
var_dep=df['instant_bookable']

X=vars_indep
y=var_dep

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['f', 't', 'f', ..., 't', 'f', 't'], dtype=object)

In [25]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[3648 1186]
 [1224 1917]]


In [26]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.7487684729064039


In [27]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.6978056426332289


In [28]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.7546545304095986


In [3]:
df["room_type"] = df["room_type"].replace({
    "Entire home/apt": 1,
    "Private room": 2,
    "Shared room": 3,
    "Hotel room": 4
})

print(df["room_type"].value_counts())

room_type
1    17363
2     8693
3      401
4      125
Name: count, dtype: int64


C:\Users\angie\AppData\Local\Temp\ipykernel_5580\4175036345.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["room_type"] = df["room_type"].replace({


In [4]:
df["bedrooms"] = df["bedrooms"].replace({
    0: "De 0 a 1 recamara",
    1: "De 0 a 1 recamara",
    1.4: "De 0 a 1 recamara",
    2: "De 2 a 3 recamaras",
    3: "De 2 a 3 recamaras"
})

# Verificar que la conversión fue correcta
print(df["bedrooms"].value_counts())

bedrooms
De 0 a 1 recamara     17625
De 2 a 3 recamaras     8957
Name: count, dtype: int64


In [5]:
vars_indep=df[['room_type', 'bathrooms', 'beds']]
var_dep=df['bedrooms']

X=vars_indep
y=var_dep

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['De 0 a 1 recamara', 'De 0 a 1 recamara', 'De 0 a 1 recamara', ...,
       'De 0 a 1 recamara', 'De 2 a 3 recamaras', 'De 2 a 3 recamaras'],
      dtype=object)

In [7]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[4530  770]
 [ 603 2072]]


In [8]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="De 0 a 1 recamara")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.8825248392752776


In [9]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.8278369905956113


In [10]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="De 0 a 1 recamara")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.8547169811320755


In [11]:
df["minimum_nights"] = df["minimum_nights"].replace({
    1: "De 0 a 2 noches minimas",
    1.6: "De 0 a 2 noches minimas",
    2: "De 2 a 3 noches minimas",
    3: "De 2 a 3 noches minimas"
})

# Verificar que la conversión fue correcta
print(df["minimum_nights"].value_counts())

minimum_nights
De 0 a 2 noches minimas    16205
De 2 a 3 noches minimas    10377
Name: count, dtype: int64


In [12]:
vars_indep=df[['price', 'room_type', 'availability_365']]
var_dep=df['minimum_nights']

X=vars_indep
y=var_dep

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['De 0 a 2 noches minimas', 'De 2 a 3 noches minimas',
       'De 0 a 2 noches minimas', ..., 'De 2 a 3 noches minimas',
       'De 0 a 2 noches minimas', 'De 0 a 2 noches minimas'], dtype=object)

In [14]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[4165  687]
 [2366  757]]


In [19]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="De 0 a 2 noches minimas")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.6377277599142551


In [20]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.6171786833855799


In [22]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="De 0 a 2 noches minimas")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.8584089035449299
